In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [95]:
train = pd.read_csv('G:/Downloads/crowdflower-search-relevance/train.csv/train.csv')
test = pd.read_csv('G:/Downloads/crowdflower-search-relevance/test.csv/test.csv')
sample = pd.read_csv('G:/Downloads/crowdflower-search-relevance/sampleSubmission.csv/sampleSubmission.csv')

In [3]:
train.shape

(10158, 6)

((6805, 4), (3353, 4), (6805,), (3353,))

In [96]:
train['product_description'].fillna('0', inplace=True)

In [97]:
train['weight'] = 1 / (1 + train['relevance_variance'])

In [21]:
train

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,0,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471
...,...,...,...,...,...,...
10153,32655,plantronics corded headset,Plantronics TriStar H81N - headset,With four sizes of user-selectable earbuds and...,4,0.471
10154,32659,spiderman,Marvel Legends Infinite Series Amazing Spider-...,0,2,0.471
10155,32663,playstation vita system,MLB 15: The Show (PlayStation Vita),An essential part of every fan’s season and Th...,2,1.247
10156,32666,flower bulbs,Junkyard Findings Vintage Trinkets-Small Typo ...,PRIMA FLOWERS-Junkyard Findings Vintage Trinke...,1,0.471


In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import scipy
import nltk
from scipy.sparse import coo_matrix, hstack

In [100]:
stop_words = set(stopwords.words('english')) 

In [101]:
train['query'] = train['query'].str.lower()
train['product_title'] = train['product_title'].str.lower()
train['product_description'] = train['product_description'].str.lower()

In [102]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [103]:
train['query'] = train['query'].apply(lemmatize_text)
train['product_title'] = train['product_title'].apply(lemmatize_text)
train['product_description'] = train['product_description'].apply(lemmatize_text)

In [104]:
def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 = str1 + ' ' + ele      
    return str1[1:]

In [105]:
train['query'] = train['query'].apply(listToString)
train['product_title'] = train['product_title'].apply(listToString)
train['product_description'] = train['product_description'].apply(listToString)

In [106]:
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1,6))
vectorizer1 = TfidfVectorizer(stop_words=stop_words, ngram_range=(1,6))
vectorizer2 = TfidfVectorizer(stop_words=stop_words, ngram_range=(1,6))

In [107]:
X = vectorizer.fit_transform(train['query'])
X1 = vectorizer1.fit_transform(train['product_title'])
X2 = vectorizer2.fit_transform(train['product_description'])

In [54]:
X.shape,X1.shape,X2.shape

((10158, 503), (10158, 157423), (10158, 1156266))

In [108]:
tr = hstack([X,X1,X2])

In [48]:
tr.shape

(10158, 36495)

In [109]:
X_train, X_test, y_train, y_test = train_test_split(tr, train['median_relevance'], 
                                                    test_size=0.3, random_state=123)

In [110]:
weight = train['weight'].iloc[y_train.index]

In [116]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7110, 1694439), (3048, 1694439), (7110,), (3048,))

In [112]:
from xgboost import XGBClassifier
from sklearn.metrics import cohen_kappa_score

In [117]:
%%time
xgb = XGBClassifier(sample_weight=weight)
xgb.fit(X_train, y_train)

[02:52:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { sample_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Wall time: 2min 6s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1,
              sample_weight=281     0.949668
9459    4.000000
347     2.039429
3613    1.359619
5408    4.000000
          ...   
9785    4.000000
7763    2.000000
5218    2.039429
1346    2.719239
3582    2.039429
Name: weight, Length: 7110, dtype: float64,
              scale_pos_weight=None, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [118]:
y_preds = xgb.predict(X_test)

In [119]:
cohen_kappa_score(y_test,y_preds,weights='quadratic')

0.353019619457401

In [76]:
cohen_kappa_score(y_test,y_preds,weights='quadratic')

0.353019619457401

In [128]:
from lightgbm import LGBMClassifier

In [135]:
%%time
lgbm = LGBMClassifier(n_jobs=-1,n_estimators=1000)
lgbm.fit(X_train, y_train)

Wall time: 3min 32s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [136]:
y_preds = lgbm.predict(X_test)
cohen_kappa_score(y_test,y_preds,weights='quadratic')

0.42858101891305944

In [69]:
from xgboost import XGBRegressor

In [24]:
%%time
xgb = XGBRegressor(n_estimators=1500, eval_metric='rmse')
xgb.fit(X_train, y_train)

Wall time: 2min 26s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1500, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [26]:
y_preds = xgb.predict(X_test)

In [29]:
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score

In [30]:
mean_squared_error(y_test,y_preds), r2_score(y_test,y_preds)

(1.0744249741728824, 0.21294389183428275)

In [31]:
y_preds

array([1.8547031, 1.8616464, 1.8666785, ..., 3.6828585, 2.9216523,
       2.5272646], dtype=float32)

In [34]:
iii = y_test.index

In [35]:
y_ts = train['median_relevance'].iloc[iii]

In [38]:
y_pr_clf = []
for i in y_preds:
    if i < 1.5:
        y_pr_clf.append(1)
    elif i < 2.5:
        y_pr_clf.append(2)
    elif i < 3.5:
        y_pr_clf.append(3)
    else:
        y_pr_clf.append(4)

In [41]:
cohen_kappa_score(y_ts,y_pr_clf,weights='quadratic')

0.3953204221946591

In [54]:
sample['prediction'] = y_preds

In [55]:
sample.to_csv('G:/Downloads/crowdflower-search-relevance/sampleSubmission.csv/sampleSubmission.csv', index=False)